<a href="https://colab.research.google.com/github/deepluckyperson/sber_homework/blob/main/%D0%94%D0%97_SQL_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [2]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [3]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [4]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [5]:
os.path.exists(path)

True

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [6]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [7]:
cursor = conn.cursor()

In [8]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [9]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [10]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [11]:
conn.commit()
conn.close()

In [12]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [13]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [14]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [15]:
cursor.close()
conn.close()

In [16]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [17]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [18]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [19]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [20]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50


In [21]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [22]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [23]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-15 12:45:22,910 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-15 12:45:22,912 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:22,916 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-15 12:45:22,918 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [24]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [25]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4543 sha256=a84a69cbd0ad30abe96627e8a74031ff754b69f3b2ed2a55c1aae850a265a21e
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [26]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [27]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [28]:
Purchase.ItemId.name

'ItemId'

In [29]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [30]:
Base.metadata.create_all(engine)

2020-12-15 12:45:28,372 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-15 12:45:28,374 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,376 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-15 12:45:28,378 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,379 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-15 12:45:28,381 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,382 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-15 12:45:28,384 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,386 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-15 12:45:28,387 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,388 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-15 12:45:28,390 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,393 INFO sqlalchemy.engine.base.Engine 
CREATE

In [31]:
engine.table_names()

2020-12-15 12:45:28,417 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-15 12:45:28,420 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [32]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-15 12:45:28,439 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:45:28,442 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 12:45:28,444 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-15 12:45:28,448 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [33]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-15 12:45:28,461 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-15 12:45:28,462 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [34]:
display_results(customers_query)

2020-12-15 12:45:28,473 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-15 12:45:28,475 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [35]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-15 12:45:28,512 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:45:28,514 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 12:45:28,516 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-15 12:45:28,518 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-15 12:45:28,529 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:45:28,532 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 12:45:28,533 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-15 12:45:28,535 INFO sqlalchemy.engine.base.Engine COMMIT


In [37]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-15 12:45:28,549 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:45:28,551 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 12:45:28,552 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-15 12:45:28,555 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [38]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-15 12:45:28,576 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-15 12:45:28,578 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [39]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf4c0>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf518>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c925fc0>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf360>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf308>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf570>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f260c8cf468>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7f25f0925630; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

In [40]:
list_info = session.query(Customer).all()

2020-12-15 12:45:28,619 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:45:28,622 INFO sqlalchemy.engine.base.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email" 
FROM customers
2020-12-15 12:45:28,623 INFO sqlalchemy.engine.base.Engine ()


In [41]:
list_info

[Customer(CustomerId=1, FirstName='Moshe', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='053-5556789', Email='moshe@cohen.com'),
 Customer(CustomerId=2, FirstName='Lisa', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='052-1234565', Email='lisa@cohen.com'),
 Customer(CustomerId=3, FirstName='Nika', LastName='Rave', Company=None, Address='Green st, LA', Phone='330-1234565', Email='Nika@rave.com'),
 Customer(CustomerId=4, FirstName='Lisa', LastName='White', Company=None, Address='Alenbi 66, Tel Aviv', Phone='062-1234565', Email='lisa@White.com')]

In [42]:
for id_ in list_info:
    print(id_.CustomerId)

1
2
3
4


In [43]:
first_info = session.query(Customer.CustomerId)

In [44]:
first_info

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [45]:
from sqlalchemy import *

In [46]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-15 12:45:28,686 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-15 12:45:28,689 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [47]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-15 12:45:28,725 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-15 12:45:28,727 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [48]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-15 12:45:28,769 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-15 12:45:28,771 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [49]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

In [50]:
display_results(customers_query)

2020-12-15 12:45:28,818 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-15 12:45:28,820 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [51]:
import pandas as pd

In [52]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [53]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-15 12:45:28,890 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-15 12:45:28,891 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,893 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-15 12:45:28,895 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,897 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-15 12:45:28,899 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [54]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-15 12:45:28,980 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-15 12:45:28,986 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,988 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-15 12:45:28,992 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:28,996 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-15 12:45:28,998 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [55]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-15 12:45:29,084 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-15 12:45:29,087 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:29,090 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-15 12:45:29,093 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 12:45:29,099 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-15 12:45:29,101 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [56]:
apple = Item(
    Name='Apple',
    Price=5
)
session.add(apple)
session.commit()

potato = Item(
    Name='Potato',
    Price=2
)
session.add(potato)
session.commit()

tomato = Item(
    Name='Tomato',
    Price=8
)
session.add(tomato)
session.commit()

pineapple = Item(
    Name='Pineapple',
    Price=20
)
session.add(pineapple)
session.commit()

million = Item(
    Name='Million',
    Price=1000000
)
session.add(million)
session.commit()

2020-12-15 12:46:48,360 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 12:46:48,361 INFO sqlalchemy.engine.base.Engine ('Apple', 5.0)
2020-12-15 12:46:48,364 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 12:46:48,366 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:46:48,368 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 12:46:48,368 INFO sqlalchemy.engine.base.Engine ('Potato', 2.0)
2020-12-15 12:46:48,370 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 12:46:48,374 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:46:48,376 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 12:46:48,377 INFO sqlalchemy.engine.base.Engine ('Tomato', 8.0)
2020-12-15 12:46:48,378 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 12:46:48,381 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:46:48,384

In [57]:
import datetime
first = Purchase(
    ItemId=1,
    CustomerId=1,
    Date=datetime.datetime.now()
)
session.add(first)
session.commit()

second = Purchase(
    ItemId=2,
    CustomerId=2,
    Date=datetime.datetime.now()
)
session.add(second)
session.commit()

third = Purchase(
    ItemId=3,
    CustomerId=3,
    Date=datetime.datetime.now()
)
session.add(third)
session.commit()

fourth = Purchase(
    ItemId=4,
    CustomerId=4,
    Date=datetime.datetime.now()
)
session.add(fourth)
session.commit()

fifth = Purchase(
    ItemId=5,
    CustomerId=4,
    Date=datetime.datetime.now()
)
session.add(fifth)
session.commit()

2020-12-15 12:47:02,240 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:47:02,245 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 12:47:02,246 INFO sqlalchemy.engine.base.Engine (1, 1, '2020-12-15 12:47:02.239345')
2020-12-15 12:47:02,248 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 12:47:02,251 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:47:02,252 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 12:47:02,253 INFO sqlalchemy.engine.base.Engine (2, 2, '2020-12-15 12:47:02.250503')
2020-12-15 12:47:02,256 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 12:47:02,258 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 12:47:02,260 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 12:47:02,261 INFO sqlalchemy.engine.base.Eng

In [58]:
customers_query = select([Customer.CustomerId,Customer.FirstName, Customer.Email])
display_results(customers_query)

items_query = select([Item.ItemId,Item.Name, Item.Price])
display_results(items_query)

purchases_query = select([Purchase.ItemId,Purchase.CustomerId, Purchase.Date, Purchase.PurchaseId])
display_results(purchases_query)

2020-12-15 12:47:16,070 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."CustomerId", customers."FirstName", customers."Email" 
FROM customers
2020-12-15 12:47:16,073 INFO sqlalchemy.engine.base.OptionEngine ()


,CustomerId,FirstName,Email
0,1,Moshe,moshe@cohen.com
1,2,Lisa,lisa@cohen.com
2,3,Nika,Nika@rave.com
3,4,Lisa,lisa@White.com


2020-12-15 12:47:16,091 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId", items."Name", items."Price" 
FROM items
2020-12-15 12:47:16,095 INFO sqlalchemy.engine.base.OptionEngine ()


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,ItemId,Name,Price
0,1,Apple,5.0
1,2,Potato,2.0
2,3,Tomato,8.0
3,4,Pineapple,20.0
4,5,Million,1000000.0


2020-12-15 12:47:16,116 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."ItemId", purchases."CustomerId", purchases."Date", purchases."PurchaseId" 
FROM purchases
2020-12-15 12:47:16,118 INFO sqlalchemy.engine.base.OptionEngine ()


,ItemId,CustomerId,Date,PurchaseId
0,1,1,2020-12-15 12:47:02.239345,1
1,2,2,2020-12-15 12:47:02.250503,2
2,3,3,2020-12-15 12:47:02.257693,3
3,4,4,2020-12-15 12:47:02.265018,4
4,5,4,2020-12-15 12:47:02.271889,5


In [61]:
from sqlalchemy.sql import text
s = text("select items.Name from items inner join purchases on purchases.ItemId==items.ItemId inner join customers on customers.CustomerId==purchases.CustomerId where customers.FirstName == :x")
conn.execute(s, x = 'Lisa').fetchall()

df = pd.read_sql("""SELECT items."Name" 
FROM purchases JOIN customers ON purchases."CustomerId"=customers."CustomerId" JOIN items ON purchases."ItemId"=items."ItemId"
WHERE customers."FirstName"='Lisa';""", conn)
print(df)

2020-12-15 13:19:58,562 INFO sqlalchemy.engine.base.Engine select items.Name from items inner join purchases on purchases.ItemId==items.ItemId inner join customers on customers.CustomerId==purchases.CustomerId where customers.FirstName == ?


INFO:sqlalchemy.engine.base.Engine:select items.Name from items inner join purchases on purchases.ItemId==items.ItemId inner join customers on customers.CustomerId==purchases.CustomerId where customers.FirstName == ?


2020-12-15 13:19:58,570 INFO sqlalchemy.engine.base.Engine ('Lisa',)


INFO:sqlalchemy.engine.base.Engine:('Lisa',)


2020-12-15 13:19:58,572 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT items.""Name"" 
FROM purchases JOIN customers ON purchases.""CustomerId""=customers.""CustomerId"" JOIN items ON purchases.""ItemId""=items.""ItemId""
WHERE customers.""FirstName""='Lisa';")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("SELECT items.""Name"" 
FROM purchases JOIN customers ON purchases.""CustomerId""=customers.""CustomerId"" JOIN items ON purchases.""ItemId""=items.""ItemId""
WHERE customers.""FirstName""='Lisa';")


2020-12-15 13:19:58,575 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 13:19:58,579 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT items.""Name"" 
FROM purchases JOIN customers ON purchases.""CustomerId""=customers.""CustomerId"" JOIN items ON purchases.""ItemId""=items.""ItemId""
WHERE customers.""FirstName""='Lisa';")


INFO:sqlalchemy.engine.base.Engine:PRAGMA temp.table_info("SELECT items.""Name"" 
FROM purchases JOIN customers ON purchases.""CustomerId""=customers.""CustomerId"" JOIN items ON purchases.""ItemId""=items.""ItemId""
WHERE customers.""FirstName""='Lisa';")


2020-12-15 13:19:58,582 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 13:19:58,586 INFO sqlalchemy.engine.base.Engine SELECT items."Name" 
FROM purchases JOIN customers ON purchases."CustomerId"=customers."CustomerId" JOIN items ON purchases."ItemId"=items."ItemId"
WHERE customers."FirstName"='Lisa';


INFO:sqlalchemy.engine.base.Engine:SELECT items."Name" 
FROM purchases JOIN customers ON purchases."CustomerId"=customers."CustomerId" JOIN items ON purchases."ItemId"=items."ItemId"
WHERE customers."FirstName"='Lisa';


2020-12-15 13:19:58,588 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


        Name
0     Potato
1  Pineapple
2    Million
